```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [1]:
from IPython.display import display, HTML
from pyspark.sql import SparkSession
from pyspark import StorageLevel
import pandas as pd
from pyspark.sql.types import StructType, StructField,StringType, LongType, IntegerType, DoubleType, ArrayType
from pyspark.sql.functions import regexp_replace
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from pyspark.sql.functions import col, split, expr
from pyspark.sql.functions import udf, lit, flatten
from pywebhdfs.webhdfs import PyWebHdfsClient
from datetime import date
from pyspark.sql.functions import monotonically_increasing_id 
import json

In [2]:
# spark.scheduler.mode', 'FAIR'
spark = SparkSession.\
    builder.\
    appName("Overpass-API").\
    enableHiveSupport().\
    master("local[*]").\
    master("spark://spark-master:7077").\
    config("spark.executor.memory", "15G").\
    config("spark.driver.maxResultSize", "135G").\
    config("spark.sql.shuffle.partitions", "500").\
    config(' spark.sql.adaptive.coalescePartitions.enabled', True).\
    config('spark.sql.adaptive.enabled', True).\
    config('spark.sql.adaptive.coalescePartitions.initialPartitionNum', 125).\
    config("spark.sql.execution.arrow.pyspark.enabled", True).\
    config("spark.sql.execution.arrow.fallback.enabled", True).\
    config('spark.kryoserializer.buffer.max', 2047).\
    config("spark.serializer", KryoSerializer.getName).\
    config("spark.kryo.registrator", SedonaKryoRegistrator.getName).\
    config("spark.jars.packages", "org.apache.sedona:sedona-python-adapter-3.0_2.12:1.1.0-incubating,org.datasyslab:geotools-wrapper:1.1.0-25.2") .\
    enableHiveSupport().\
    getOrCreate()

SedonaRegistrator.registerAll(spark)
sc = spark.sparkContext

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.sedona#sedona-python-adapter-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c20579c1-d499-4129-ae06-0876b2e0525a;1.0
	confs: [default]
	found org.apache.sedona#sedona-python-adapter-3.0_2.12;1.1.0-incubating in central
	found org.locationtech.jts#jts-core;1.18.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found com.fasterxml.jackson.core#jackson-databind;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.12.2 in central
	found com.fasterxml.jackson.core#jackson-core;2.12.2 in central
	found org.apache.sedona#sedona-core-3.0_2.12;1.1.0-incubating in central
	found org.apache.sedona#sedona-sql

In [3]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
# overpass_query = """
# [out:json];
# area["ISO3166-1"="DE"][admin_level=2];
# (node["amenity"="biergarten"](area);
#  way["amenity"="biergarten"](area);
#  rel["amenity"="biergarten"](area);
# );
# out center;
# """

# overpass_query = """
# [out:json];
# area[name = "Foz do Iguaçu"];
# (way(area)["highway"~"^(private|cycleway|footway|bus_guideway|elevator|construction|proposed|bridleway|steps|raceway|motorway_link|path|secondary|motorway|trunk|primary)$"];>;);
# out center;
# """

overpass_query = """
[out:json];
area[name = "Foz do Iguaçu"];
way(area)["highway"~""];
out geom;
>;
out skel qt;
"""

# response = requests.get(overpass_url, 
#                         params={'data': overpass_query})
# data = response.json()
# hdfs = PyWebHdfsClient(host='179.106.229.159',port='50070', user_name='root')
file_name = "foz_roads_osm.json"
# hdfs.delete_file_dir(file_name)
# hdfs.create_file(file_name, json.dumps(data))

In [4]:
path = "hdfs://776faf4d6a1e:8020/"+file_name
df = spark.read.json(path, multiLine = "true")

22/02/01 12:44:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
df.printSchema()

root
 |-- elements: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- bounds: struct (nullable = true)
 |    |    |    |-- maxlat: double (nullable = true)
 |    |    |    |-- maxlon: double (nullable = true)
 |    |    |    |-- minlat: double (nullable = true)
 |    |    |    |-- minlon: double (nullable = true)
 |    |    |-- geometry: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- lat: double (nullable = true)
 |    |    |    |    |-- lon: double (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lon: double (nullable = true)
 |    |    |-- nodes: array (nullable = true)
 |    |    |    |-- element: long (containsNull = true)
 |    |    |-- tags: struct (nullable = true)
 |    |    |    |-- FIXME: string (nullable = true)
 |    |    |    |-- access: string (nullable = true)
 |    |    |    |-- addr:city: string (nullabl

In [6]:
from pyspark.sql.functions import explode, arrays_zip

df.createOrReplaceTempView("df")
tb = spark.sql("select *, size(elements) total_nodes from df")
tb.show(5)

isolate_total_nodes = tb.select("total_nodes").toPandas()
total_nodes = isolate_total_nodes["total_nodes"].iloc[0]
print(total_nodes)

isolate_ids = tb.select("elements.id").toPandas()
ids = pd.DataFrame(isolate_ids["id"].iloc[0]).drop_duplicates()
print(ids[0].iloc[1])

formatted_df = tb\
.withColumn("id", explode("elements.id"))

formatted_df.show(5)

formatted_df = tb\
.withColumn("new", arrays_zip("elements.id", "elements.geometry", "elements.nodes", "elements.tags"))\
.withColumn("new", explode("new"))

formatted_df.show(5)

# formatted_df.printSchema()

formatted_df = formatted_df.select("new.0","new.1","new.2","new.3.maxspeed","new.3.incline","new.3.surface", "new.3.name", "total_nodes")
formatted_df = formatted_df.withColumnRenamed("0","id").withColumnRenamed("1","geom").withColumnRenamed("2","nodes").withColumnRenamed("3","tags")
formatted_df.createOrReplaceTempView("formatted_df")
formatted_df.show(5)
# TODO atualizar daqui para baixo para considerar a linha inteira na lógica
points_tb = spark.sql("select geom, id from formatted_df where geom IS NOT NULL")
points_tb = points_tb\
.withColumn("new", arrays_zip("geom.lat", "geom.lon"))\
.withColumn("new", explode("new"))

points_tb = points_tb.select("new.0","new.1", "id")

points_tb = points_tb.withColumnRenamed("0","lat").withColumnRenamed("1","lon")
points_tb.printSchema()

points_tb.createOrReplaceTempView("points_tb")

points_tb.show(5)

# teste = spark.sql("select st_point(lat, lon) as geom, id from points_tb")

# teste = spark.sql("select collect_list(array(p1.lat,p1.lon)) as line from points_tb p1 where p1.id = 25835738 group by p1.id")
# teste.show(5)
# print(teste.take(1))

# SELECT ST_AsText(ST_Envelope(
# 		ST_Collect(
# 			ST_GeomFromText('LINESTRING(55 75,125 150)'),
# 				ST_Point(20, 80))
# 				)) As wktenv;

coordinates_tb = spark.sql("select (select collect_list(CONCAT(p1.lat,',',p1.lon)) from points_tb p1 where p1.id = p2.id group by p1.id) as coordinates, p2.id, p2.maxspeed, p2.incline, p2.surface, p2.name, p2.nodes, p2.total_nodes from formatted_df p2")
coordinates_tb.createOrReplaceTempView("coordinates_tb")
coordinates_tb.show(5)

roads_tb = spark.sql("SELECT ST_LineStringFromText(REPLACE(REPLACE(CAST(coordinates as string),'[',''),']',''), ',') as geom, id, maxspeed, incline, surface, name, nodes, total_nodes FROM coordinates_tb WHERE coordinates IS NOT NULL")
roads_tb.createOrReplaceTempView("roads_tb")
roads_tb.show(5)

+--------------------+--------------------+--------------------+-------+-----------+
|            elements|           generator|               osm3s|version|total_nodes|
+--------------------+--------------------+--------------------+-------+-----------+
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|
+--------------------+--------------------+--------------------+-------+-----------+

36560
26122619


+--------------------+--------------------+--------------------+-------+-----------+--------+
|            elements|           generator|               osm3s|version|total_nodes|      id|
+--------------------+--------------------+--------------------+-------+-----------+--------+
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|25835738|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|26122619|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|26122631|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|26122645|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|26122801|
+--------------------+--------------------+--------------------+-------+-----------+--------+
only showing top 5 rows



+--------------------+--------------------+--------------------+-------+-----------+--------------------+
|            elements|           generator|               osm3s|version|total_nodes|                 new|
+--------------------+--------------------+--------------------+-------+-----------+--------------------+
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|[25835738, [[-25....|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|[26122619, [[-25....|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|[26122631, [[-25....|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|[26122645, [[-25....|
|[[[-25.5267745, -...|Overpass API 0.7....|[The data include...|    0.6|      36560|[26122801, [[-25....|
+--------------------+--------------------+--------------------+-------+-----------+--------------------+
only showing top 5 rows

+--------+-----------

In [7]:

print(roads_tb.select('geom').take(1))

[Row(geom=<shapely.geometry.linestring.LineString object at 0x7f8b6c4fefd0>)]


In [8]:
# Não foi considerado que um caminha pode necessitar mais de 1 rua

start_point = "-25.4695946,-54.5909028"
end_point = "-25.4786993,-54.57938" 

distance_tb = spark.sql("select nodes, st_distance(geom, st_point("+end_point+")) as distance_toend, st_distance(st_point("+start_point+"), geom) as distance, st_length(geom) * 1000 as geomsize, geom, id, maxspeed, incline, surface, name , total_nodes from roads_tb")
distance_tb.createOrReplaceTempView("distance_tb")
distance_tb.show(5)

# considerar distância, direcao(ex: 0 180 e etc), inclinacao(up, down, 0%), superficie(asphalt,paved, concrete), velocidade(60 80 50 40 e etc)
fill_null_tb = spark.sql(
    "select nodes, IFNULL(maxspeed, 20) as maxspeed, IFNULL(incline, '0%') as incline, IFNULL(surface, 'soil') as surface, name, id, geom, geomsize, distance, distance_toend, total_nodes  from distance_tb")
fill_null_tb.createOrReplaceTempView("fill_null_tb")
fill_null_tb.show(5)

surface_index_tb = spark.sql(
    "select nodes, case surface when 'asphalt'" +
    "then 0.01 when 'concrete'" + 
    "then 0.02 when 'paved'" +
    "then 0.03 when 'soil'" +
    "then 0.04 when 'unpaved'" + 
    "then 0.04 when 'sett'" +
    "then 0.03 ELSE 0.05 end as surface_index,"+ 
    "maxspeed, incline, surface, name, id, geom, geomsize, distance, distance_toend, total_nodes  from fill_null_tb")
surface_index_tb.createOrReplaceTempView("surface_index_tb")
surface_index_tb.show(5)

incline_index_tb = spark.sql(
    "select nodes, case incline when 'top' then -0.10 when 'down' then 0.10 when '0%' then 0 end as incline_index, surface_index, maxspeed, incline, surface, name, id, geom, geomsize, distance, distance_toend, total_nodes  from surface_index_tb")
incline_index_tb.createOrReplaceTempView("incline_index_tb")
incline_index_tb.show(5)
  
weight_index_tb = spark.sql(
    "select nodes, (maxspeed - (maxspeed * surface_index)) + (maxspeed +(maxspeed * incline_index)) as weight, incline_index, surface_index, maxspeed, incline, surface, name, id, geom, geomsize, distance, distance_toend, total_nodes  from incline_index_tb WHERE geomsize IS NOT NULL")
weight_index_tb.createOrReplaceTempView("weight_index_tb")
weight_index_tb.show(5)

+--------------------+-------------------+-------------------+------------------+--------------------+--------+--------+-------+--------+--------------------+-----------+
|               nodes|     distance_toend|           distance|          geomsize|                geom|      id|maxspeed|incline| surface|                name|total_nodes|
+--------------------+-------------------+-------------------+------------------+--------------------+--------+--------+-------+--------+--------------------+-----------+
|[362528326, 28597...|0.04960428437877076|0.06191383729684203|11.818939546233448|LINESTRING (-25.5...|25835738|    null|   null|    null|Avenida Costa e S...|      36560|
|[285975617, 59202...| 0.1036513492683037|0.11504041822963577|  8.00531493113018|LINESTRING (-25.5...|26122619|      60|   null|   paved|    Avenida Mercosul|      36560|
|[285975617, 69310...|0.10941553855472455|0.12057253982300521|1.9538901610218935|LINESTRING (-25.5...|26122631|      60|   null|concrete|Ponte In

In [9]:
teste = spark.sql(
    "select min(weight) from weight_index_tb")
teste.show(5)

+-----------+
|min(weight)|
+-----------+
|       39.0|
+-----------+



In [10]:
closestoend_tb = spark.sql(
    "select w1.id, w1.distance_toend from weight_index_tb w1 group by w1.id, w1.distance_toend having (select min(w2.distance_toend) as distance_toend  from weight_index_tb w2) = w1.distance_toend")
closestoend_tb.createOrReplaceTempView("closestoend_tb")
closestoend = closestoend_tb.take(1)[0]['id']
print(closestoend)

closestostart_tb = spark.sql(
    "select w1.id, w1.distance from weight_index_tb w1 group by w1.id, w1.distance having (select min(w2.distance) as distance  from weight_index_tb w2) = w1.distance")
closestostart_tb.createOrReplaceTempView("closestostart_tb")
closestostart_tb.show(5)

closestostart = closestostart_tb.take(1)[0]['id']

# FOLIUM EM 3857 dado em 4326 st_transform(st_union_aggr(geom),'epsg:3857','epsg:4326')
json_lines = spark.sql(
    "select ST_AsGeoJSON(st_envelope_aggr(geom)) AS json  from weight_index_tb where id in ("+str(closestostart)+","+str(closestoend)+")")
json_lines_string_teste  = json_lines.take(1)[0]['json']
coordinates_teste = json.loads(json_lines_string_teste)['coordinates']


# st_boundary st_contains

# Pegar o limite entre a uniao da geom inicial e final
# select st_boundary(st_union_aggr(geom)) AS boundary  from weight_index_tb where id in ("+str(closestostart)+","+str(closestoend)+")

boundary_tb = spark.sql("select st_envelope_aggr(geom) as boundary from weight_index_tb where id in ("+str(closestostart)+","+str(closestoend)+")")
boundary_tb.createOrReplaceTempView("boundary_tb")
boundary_tb.show(5)

contains_tb = spark.sql("select st_intersects(boundary,geom) as contains, id from weight_index_tb, boundary_tb")
contains_tb.createOrReplaceTempView("contains_tb")
contains_tb.show(5)

possible_paths = spark.sql("select id, geom from weight_index_tb group by id, geom having id in (select id from contains_tb where contains = true)")
possible_paths.createOrReplaceTempView("possible_paths")
possible_paths.show(5)

paths_collection = spark.sql("select ST_AsGeoJSON(st_union_aggr(geom)) AS json from possible_paths")
json_lines_string  = paths_collection.take(1)[0]['json']
coordinates = json.loads(json_lines_string)['coordinates']

32344883


22/02/01 12:44:21 ERROR Utils: Uncaught exception in thread element-tracking-store-worker
java.util.NoSuchElementException: key not found: 4363
	at scala.collection.MapLike.default(MapLike.scala:235)
	at scala.collection.MapLike.default$(MapLike.scala:234)
	at scala.collection.AbstractMap.default(Map.scala:63)
	at scala.collection.MapLike.apply(MapLike.scala:144)
	at scala.collection.MapLike.apply$(MapLike.scala:143)
	at scala.collection.AbstractMap.apply(Map.scala:63)
	at org.apache.spark.sql.execution.ui.SQLAppStatusListener.$anonfun$aggregateMetrics$11(SQLAppStatusListener.scala:257)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.mutable.HashMap.$anonfun$foreach$1(HashMap.scala:149)
	at scala.collection.mutable.HashTable.foreachEntry(HashTable.scala:237)
	at scala.collection.mutable.HashTable.foreachEntry$(HashTable.scala:230)
	at scala.collection.mutable.HashMap.foreachEntry(HashMap.scala:44)
	at scala.collection.mutable.HashMap.

+--------+--------------------+
|      id|            distance|
+--------+--------------------+
|32347167|8.330276767022637E-5|
+--------+--------------------+



22/02/01 12:44:22 ERROR Utils: Uncaught exception in thread element-tracking-store-worker
java.util.NoSuchElementException: key not found: 5357
	at scala.collection.MapLike.default(MapLike.scala:235)
	at scala.collection.MapLike.default$(MapLike.scala:234)
	at scala.collection.AbstractMap.default(Map.scala:63)
	at scala.collection.MapLike.apply(MapLike.scala:144)
	at scala.collection.MapLike.apply$(MapLike.scala:143)
	at scala.collection.AbstractMap.apply(Map.scala:63)
	at org.apache.spark.sql.execution.ui.SQLAppStatusListener.$anonfun$aggregateMetrics$11(SQLAppStatusListener.scala:257)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.mutable.HashMap.$anonfun$foreach$1(HashMap.scala:149)
	at scala.collection.mutable.HashTable.foreachEntry(HashTable.scala:237)
	at scala.collection.mutable.HashTable.foreachEntry$(HashTable.scala:230)
	at scala.collection.mutable.HashMap.foreachEntry(HashMap.scala:44)
	at scala.collection.mutable.HashMap.

+--------------------+
|            boundary|
+--------------------+
|POLYGON ((-25.479...|
+--------------------+

+--------+--------+
|contains|      id|
+--------+--------+
|   false|25835738|
|   false|26122619|
|   false|26122631|
|   false|26122645|
|   false|26122801|
+--------+--------+
only showing top 5 rows

+---------+--------------------+
|       id|                geom|
+---------+--------------------+
| 32344891|LINESTRING (-25.4...|
|197591238|LINESTRING (-25.4...|
|918520467|LINESTRING (-25.4...|
| 32345349|LINESTRING (-25.4...|
|964560305|LINESTRING (-25.4...|
+---------+--------------------+
only showing top 5 rows



In [11]:
path = [closestostart]
visited = [closestostart]
current_nodes = spark.sql("select geom from weight_index_tb where id = "+str(closestostart))
row = current_nodes.rdd.collect()[0]["geom"]
id_current = closestostart

def choose_path(row, path, id_current, visited, copy_row):

    visited_frm = str(visited).replace("[","(").replace("]",")")
  
    touches_tb = spark.sql("select st_touches(st_geomfromwkt('"+str(row)+"'),geom) as touches, * from weight_index_tb where geom IS NOT NULL and distance_toend IS NOT NULL")
    touches_tb.createOrReplaceTempView("touches_tb")
# st_distance(st_geomfromwkt('"+str(row)+"'),geom)
    fim_distance = spark.sql("select distance_toend from touches_tb where id = "+ str(id_current))
    fim_distance.show(5)
    fim_distance_value = fim_distance.rdd.collect()[0]["distance_toend"]
    
    
    current_distance = spark.sql("select distance from touches_tb where id = " + str(id_current))
    current_distance_value = current_distance.rdd.collect()[0]["distance"]
    
#     st_distance(st_geomfromwkt('"+str(row)+"'),geom) = 

    sql = "select geom, id, weight from touches_tb where " \
    +"touches = true" \
    +" and " \
    +"distance_toend < " \
    +str(fim_distance_value) \
    +" and " \
    +"distance > " \
    +str(current_distance_value) \
    +" and " \
    +"id NOT IN " \
    +visited_frm \
    
    print(sql)
    current_nodes = spark.sql(sql)
    current_nodes.createOrReplaceTempView("current_nodes")
    current_nodes.show(5)
    
    current_node = spark.sql("select id, geom, max(weight) from current_nodes group by id, geom, weight having max(weight) = weight ")
    current_node.show(5)
    
    if len(current_nodes.rdd.collect()) == 0:
        return path
    else:
        row = current_node.rdd.collect()[0]["geom"]
        id_current = current_node.rdd.collect()[0]["id"]
        path.append(id_current)
        visited.append(id_current)
        return choose_path(row, path, id_current, visited, copy_row)
    
path_ids = choose_path(row, path, id_current, visited, row)
path_ids.append(closestoend)
path_ids_frm = str(path_ids).replace("[","(").replace("]",")")
print(path_ids_frm)

short_path = spark.sql("select ST_AsGeoJSON(st_union_aggr(geom)) AS json from weight_index_tb where id in "+path_ids_frm)
short_path_string  = short_path.take(1)[0]['json']
short_path_coordinates = json.loads(short_path_string)['coordinates']

+-------------------+
|     distance_toend|
+-------------------+
|0.01362215583892774|
+-------------------+

select geom, id, weight from touches_tb where touches = true and distance_toend < 0.01362215583892774 and distance > 8.330276767022637e-05 and id NOT IN (32347167)
+--------------------+---------+------+
|                geom|       id|weight|
+--------------------+---------+------+
|LINESTRING (-25.4...| 32347009|  39.2|
|LINESTRING (-25.4...|197591238|  39.2|
+--------------------+---------+------+

+---------+--------------------+-----------+
|       id|                geom|max(weight)|
+---------+--------------------+-----------+
|197591238|LINESTRING (-25.4...|       39.2|
| 32347009|LINESTRING (-25.4...|       39.2|
+---------+--------------------+-----------+

+--------------------+
|      distance_toend|
+--------------------+
|0.011868762484771467|
+--------------------+

select geom, id, weight from touches_tb where touches = true and distance_toend < 0.0118687624847

In [12]:
#

import folium

start_point_arr = [-25.4695946,-54.5909028]
end_point_arr = [-25.4786993,-54.57938] 
tooltip = "Click me!"
# 3857
m = folium.Map(location=[-25.5172662,-54.6170038], zoom_start=12, tiles='OpenStreetMap', crs='EPSG3857' )
folium.Marker(
    start_point_arr, popup="<i>Inicio</i>", tooltip=tooltip, icon=folium.Icon(color="green")
).add_to(m)
folium.Marker(
    end_point_arr, popup="<b>Fim</b>", tooltip=tooltip, icon=folium.Icon(color="red")
).add_to(m)

# lines = folium.vector_layers.PolyLine(locations=coordinates)
# lines.add_to(m)

# polygon = folium.vector_layers.Polygon(locations=coordinates_teste)
# polygon.add_to(m)

polygon_path = folium.vector_layers.Polygon(locations=short_path_coordinates)
polygon_path.add_to(m)

m

In [13]:
spark.stop()